In [1]:
import numpy as np
import pandas as pd
from planar_utils import (
    plot_decision_boundary,
    sigmoid,
    load_planar_dataset,
    load_extra_datasets,
)

In [2]:
X, Y = load_planar_dataset()

In [3]:
shape_X = X.shape
shape_Y = Y.shape
m = shape_X[1] 

print("The shape of X is: " + str(shape_X))
print("The shape of Y is: " + str(shape_Y))
print("I have m = %d training examples!" % (m))

The shape of X is: (2, 400)
The shape of Y is: (1, 400)
I have m = 400 training examples!


In [4]:
def layer_size(X,Y):

    n_x = X.shape[0]
    n_y = Y.shape[0]
    n_h = 4
    return n_x, n_h, n_y


In [5]:
def define_parameters(n_x,n_h,n_y):


    scaling_factor=0.01
    w1 = np.random.randn(n_h,n_x)*scaling_factor
    w2 = np.random.randn(n_y,n_h)*scaling_factor
    b1 = np.zeros((n_h,1))
    b2 = np.zeros((n_y,1))

    parameters = { "w1" : w1 , "w2": w2 , "b1" : b1, "b2" : b2  }
    return parameters

In [6]:
n_x, n_h, n_y = layer_size(X, Y)
parameters = define_parameters(n_x, n_h, n_y)


In [7]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [8]:
def forward_prop(X, parameters):

    w1 = parameters.get("w1")
    b1 = parameters.get("b1")
    w2 = parameters.get("w2")
    b2 = parameters.get("b2")

    Z1 = np.dot(w1,X)+b1
    A1 = np.tanh(Z1)
    Z2 = np.dot(w2,A1)+b2
    A2 = 1 / (1 + np.exp(-Z2))

    cache = {"Z1" : Z1 , "A1" : A1 , "Z2" : Z2 , "A2" : A2}

    return A2 ,cache 

In [9]:
def compute_cost(A2, Y, parameters):
   
    m = Y.shape[1] 

    
    logprobs = np.multiply(np.log(A2), Y) + np.multiply((1 - Y), np.log(1 - A2))
    cost = (-1 / m) * np.sum(logprobs)
    

    cost = float(np.squeeze(cost))  # makes sure cost is the dimension we expect.
    
    assert isinstance(cost, float)

    return cost

In [10]:
def Back_prop(X , Y , cache , parameters):

    w1 = parameters.get("w1")
    b1 = parameters.get("b1")
    w2 = parameters.get("w2")
    b2 = parameters.get("b2")

    A2 = cache.get("A2")
    A1 = cache.get("A1")
    Z1 = cache.get("Z1")
    Z2 = cache.get("Z2")

    d_Z2 = A2 - Y
    d_W2 = (1/m)*np.dot(d_Z2,A1.T)
    d_b2 =  (1/m)*np.sum(d_Z2,axis=1, keepdims=True)

    d_Z1 = w2.T * d_Z2 * ( 1 - np.power(A1, 2))
    d_W1 = (1/m)*np.dot(d_Z1,X.T)
    d_b1 = (1/m)*np.sum(d_Z1)

    grads = {"dW1": d_W1, "db1": d_b1, "dW2": d_W2, "db2": d_b2}

    return grads


    

In [11]:
def update_para(grads , parameters , learning_rate=1.02):

    w1 = parameters.get("w1")
    b1 = parameters.get("b1")
    w2 = parameters.get("w2")
    b2 = parameters.get("b2")

    d_W1 = grads.get("dW1")
    d_b1 = grads.get("db1")
    d_W2 = grads.get("dW2")
    d_b2 = grads.get("db2")

    w1 -= learning_rate*d_W1
    b1 -= learning_rate*d_b1
    w2 -= learning_rate*d_W2
    b2 -= learning_rate*d_b2

    parameters = {"w1": w1, "b1": b1, "w2": w2, "b2": b2}

    return parameters


In [12]:
def nn_model(X , Y , n_h , parameters , learning_rate=1.02,iterations=10000,  print_cost=False):

    n_x = layer_size(X,Y)[0]
    n_y = layer_size(X,Y)[2]

    w1 = parameters.get("w1")
    b1 = parameters.get("b1")
    w2 = parameters.get("w2")
    b2 = parameters.get("b2")

    for i in range(0 , iterations):

        A2,cache = forward_prop(X,parameters)

        cost = compute_cost(A2,Y,parameters)

        grads = Back_prop(X , Y , cache , parameters)

        parameters = update_para(grads , parameters , learning_rate=1.02)

        if print_cost and i % 1000 == 0:
            print("Cost after iteration %i: %f" % (i, cost))

    return parameters


In [13]:
def predict(parameters, X):

    A2, _ = forward_prop(X, parameters)  # unpack tuple
    predictions = A2 > 0.5
    return predictions


In [14]:
parameters = nn_model(X, Y, n_h,parameters,learning_rate=1.02, iterations=10000, print_cost=True)


Cost after iteration 0: 0.693113
Cost after iteration 1000: 0.290080
Cost after iteration 2000: 0.276785


Cost after iteration 3000: 0.268790
Cost after iteration 4000: 0.263289
Cost after iteration 5000: 0.259274
Cost after iteration 6000: 0.259974
Cost after iteration 7000: 0.259560
Cost after iteration 8000: 0.226745
Cost after iteration 9000: 0.227836


In [15]:
# Assuming 'parameters' is already trained, and 'X' and 'Y' are the dataset
predictions = predict(parameters, X)
accuracy = float((np.dot(Y, predictions.T) + np.dot(1 - Y, 1 - predictions.T)) / Y.size * 100)
print(f"Accuracy: {accuracy:.2f}%")


Accuracy: 90.00%


C:\Users\manas\AppData\Local\Temp\ipykernel_13820\3497124417.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  accuracy = float((np.dot(Y, predictions.T) + np.dot(1 - Y, 1 - predictions.T)) / Y.size * 100)
